# 第10章 数据聚合与分组操作

## 1.GroupBy机制

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.DataFrame({'key1': ['a','a','b','b','a'],
                   'key2': ['one','two','one','two','one'],
                   'data1': np.random.randn(5),
                   'data2': np.random.randn(5)})

In [3]:
df

,key1,key2,data1,data2
0,a,one,1.372041,0.008172
1,a,two,0.829671,0.712297
2,b,one,-0.501553,-0.872887
3,b,two,0.692401,0.939431
4,a,one,0.981371,-0.211836


In [6]:
grouped = df['data1'].groupby(df['key1'])

In [7]:
grouped

In [8]:
grouped.mean()

key1
a    0.328920
b    0.021762
Name: data1, dtype: float64

In [10]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()

In [11]:
means

key1  key2
a     one     0.866184
      two    -0.745607
b     one     0.266448
      two    -0.222924
Name: data1, dtype: float64

In [14]:
states = np.array(['Ohio','California','California','Ohio','Ohio'])
years = np.array([2005,2005,2006,2005,2006])

In [15]:
df['data1'].groupby([states,years]).mean()

California  2005   -0.745607
            2006    0.266448
Ohio        2005    0.514786
            2006    0.479871
Name: data1, dtype: float64

In [16]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,0.328920,-1.477453
b,0.021762,-0.367860


In [17]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   0.866184 -0.900640
     two  -0.745607 -2.631080
b    one   0.266448  0.272181
     two  -0.222924 -1.007900

### 1.1遍历各分组

In [20]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  1.252496 -1.046922
1    a  two -0.745607 -2.631080
4    a  one  0.479871 -0.754358
b
  key1 key2     data1     data2
2    b  one  0.266448  0.272181
3    b  two -0.222924 -1.007900


In [21]:
for (k1, k2), group in df.groupby(['key1','key2']):
    print(k1, k2)
    print(group)

a one
  key1 key2     data1     data2
0    a  one  1.252496 -1.046922
4    a  one  0.479871 -0.754358
a two
  key1 key2     data1    data2
1    a  two -0.745607 -2.63108
b one
  key1 key2     data1     data2
2    b  one  0.266448  0.272181
b two
  key1 key2     data1   data2
3    b  two -0.222924 -1.0079


### 1.2选择一列或所有列的子集

In [23]:
df.groupby(['key1','key2'])[['data2']].mean()

data2
key1 key2          
a    one  -0.900640
     two  -2.631080
b    one   0.272181
     two  -1.007900

### 1.3 使用字典和Series分组

In [24]:
people = pd.DataFrame(np.random.randn(5,5),
                     columns = ['a','b','c','d','e'],
                     index = ['Joe','Steve','Wes','Jim','Travis'])

In [25]:
people.iloc[2:3, [1,2]] = np.nan

In [26]:
people

,a,b,c,d,e
Joe,0.482605,-2.065416,1.184947,-0.736176,-2.610424
Steve,-0.113483,0.493184,-0.225800,-1.611596,-1.904243
Wes,-0.241520,NaN,NaN,1.391570,-1.200500
Jim,-0.751878,-1.304132,-0.758132,-0.662834,0.091113
Travis,0.287342,0.346399,2.254853,-0.667448,-0.104422


假设有各列的分组对应关系，现准备将各列按组累加

In [27]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
           'd': 'blue', 'e': 'red', 'f': 'orange'}

In [28]:
by_column = people.groupby(mapping, axis = 1)

In [29]:
by_column.sum()

,blue,red
Joe,0.448772,-4.193234
Steve,-1.837395,-1.524543
Wes,1.391570,-1.442020
Jim,-1.420966,-1.964898
Travis,1.587405,0.529318


In [30]:
map_series = pd.Series(mapping)

In [31]:
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [32]:
people.groupby(map_series, axis = 1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 1.4使用函数分组

In [33]:
people.groupby(len).sum()

,a,b,c,d,e
3,-0.510793,-3.369548,0.426815,-0.007440,-3.719812
5,-0.113483,0.493184,-0.225800,-1.611596,-1.904243
6,0.287342,0.346399,2.254853,-0.667448,-0.104422


### 1.5根据索引层级分组

In [35]:
columns = pd.MultiIndex.from_arrays([['US','US','US','JP','JP'],
                                    [1,3,5,1,3]],
                                    names = ['city','tenor'])

In [36]:
hier_df = pd.DataFrame(np.random.randn(4,5), columns = columns)

In [37]:
hier_df

city         US                            JP          
tenor         1         3         5         1         3
0      2.040102  1.083541 -0.708933  0.543138 -0.016532
1      1.092438 -0.158867  0.430549  0.902867 -0.315464
2     -1.150629  0.672267 -1.700302 -1.668971  0.150616
3     -0.980395  0.375573 -0.026931 -1.861526 -1.160229

In [39]:
hier_df.groupby(level='city', axis = 1).count()

city,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 2.数据聚合

In [4]:
df

,key1,key2,data1,data2
0,a,one,1.372041,0.008172
1,a,two,0.829671,0.712297
2,b,one,-0.501553,-0.872887
3,b,two,0.692401,0.939431
4,a,one,0.981371,-0.211836


In [5]:
grouped = df.groupby('key1')

In [6]:
grouped['data1'].quantile(0.9)

key1
a    1.293907
b    0.573005
Name: data1, dtype: float64

In [7]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

In [8]:
grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,0.542371,0.924134
b,1.193954,1.812317


In [9]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  1.061028  0.279822  0.829671  0.905521  0.981371  1.176706   
b      2.0  0.095424  0.844253 -0.501553 -0.203065  0.095424  0.393912   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     1.372041   3.0  0.169544  0.482739 -0.211836 -0.101832  0.008172   
b     0.692401   2.0  0.033272  1.281502 -0.872887 -0.419807  0.033272   

                          
           75%       max  
key1                      
a     0.360234  0.712297  
b     0.486351  0.939431

### 2.1逐级及多函数应用

In [10]:
import os
os.chdir('C:/Users/Van/Desktop/desktop file/pydata-book-2nd-edition/examples')

In [11]:
tips = pd.read_csv('tips.csv')

In [12]:
tips['tip_pct'] = tips['tip'] - tips['total_bill']

In [13]:
tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,-15.98
1,10.34,1.66,No,Sun,Dinner,3,-8.68
2,21.01,3.50,No,Sun,Dinner,3,-17.51
3,23.68,3.31,No,Sun,Dinner,2,-20.37
4,24.59,3.61,No,Sun,Dinner,4,-20.98
5,25.29,4.71,No,Sun,Dinner,4,-20.58


In [14]:
grouped = tips.groupby(['day', 'smoker'])

In [15]:
grouped_pct = grouped['tip_pct']

In [16]:
grouped_pct.agg('mean')

day   smoker
Fri   No       -15.607500
      Yes      -14.099333
Sat   No       -16.558889
      Yes      -18.401190
Sun   No       -17.338772
      Yes      -20.603158
Thur  No       -14.439333
      Yes      -16.160588
Name: tip_pct, dtype: float64

In [17]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean        std  peak_to_peak
day  smoker                                    
Fri  No     -15.607500   4.285474          8.54
     Yes    -14.099333   8.289421         30.69
Sat  No     -16.558889   7.542277         35.29
     Yes    -18.401190   9.252859         39.73
Sun  No     -17.338772   7.268577         36.40
     Yes    -20.603158  10.628917         39.75
Thur No     -14.439333   6.684539         30.68
     Yes    -16.160588   7.564381         29.77

In [18]:
grouped_pct.agg([('foo', 'mean'),('bar', np.std)])

foo        bar
day  smoker                      
Fri  No     -15.607500   4.285474
     Yes    -14.099333   8.289421
Sat  No     -16.558889   7.542277
     Yes    -18.401190   9.252859
Sun  No     -17.338772   7.268577
     Yes    -20.603158  10.628917
Thur No     -14.439333   6.684539
     Yes    -16.160588   7.564381

In [22]:
functions = ['count', 'mean', 'max']

In [23]:
result = grouped['tip_pct', 'total_bill'].agg(functions)

In [24]:
result

tip_pct                   total_bill                  
              count       mean    max      count       mean    max
day  smoker                                                       
Fri  No           4 -15.607500 -10.96          4  18.420000  22.75
     Yes         15 -14.099333  -4.75         15  16.813333  40.17
Sat  No          45 -16.558889  -6.25         45  19.661778  48.33
     Yes         42 -18.401190  -2.07         42  21.276667  50.81
Sun  No          57 -17.338772  -6.77         57  20.506667  48.17
     Yes         19 -20.603158  -2.10         19  24.120000  45.35
Thur No          45 -14.439333  -5.51         45  17.113111  41.19
     Yes         17 -16.160588  -8.34         17  19.190588  43.11

In [25]:
result['tip_pct']

count       mean    max
day  smoker                         
Fri  No          4 -15.607500 -10.96
     Yes        15 -14.099333  -4.75
Sat  No         45 -16.558889  -6.25
     Yes        42 -18.401190  -2.07
Sun  No         57 -17.338772  -6.77
     Yes        19 -20.603158  -2.10
Thur No         45 -14.439333  -5.51
     Yes        17 -16.160588  -8.34

### 2.2返回不含行索引的聚合数据

In [26]:
tips.groupby(['day', 'smoker'], as_index = False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,-15.607500
1,Fri,Yes,16.813333,2.714000,2.066667,-14.099333
2,Sat,No,19.661778,3.102889,2.555556,-16.558889
3,Sat,Yes,21.276667,2.875476,2.476190,-18.401190
4,Sun,No,20.506667,3.167895,2.929825,-17.338772
5,Sun,Yes,24.120000,3.516842,2.578947,-20.603158
6,Thur,No,17.113111,2.673778,2.488889,-14.439333
7,Thur,Yes,19.190588,3.030000,2.352941,-16.160588


### 3应用：通用拆分——应用——联合

In [27]:
def top(df, n=5, column='tip_pct'):
    return df.sort_values(by = column)[-n:]

In [28]:
top(tips, n=6)

,total_bill,tip,smoker,day,time,size,tip_pct
195,7.56,1.44,No,Thur,Lunch,2,-6.12
178,9.60,4.00,Yes,Sun,Dinner,2,-5.60
149,7.51,2.00,No,Thur,Lunch,2,-5.51
92,5.75,1.00,Yes,Fri,Dinner,2,-4.75
172,7.25,5.15,Yes,Sun,Dinner,2,-2.10
67,3.07,1.00,Yes,Sat,Dinner,1,-2.07


In [29]:
tips.groupby('smoker').apply(top)

total_bill   tip smoker   day    time  size  tip_pct
smoker                                                          
No     145        8.35  1.50     No  Thur   Lunch     2    -6.85
       6          8.77  2.00     No   Sun  Dinner     2    -6.77
       111        7.25  1.00     No   Sat  Dinner     1    -6.25
       195        7.56  1.44     No  Thur   Lunch     2    -6.12
       149        7.51  2.00     No  Thur   Lunch     2    -5.51
Yes    218        7.74  1.44    Yes   Sat  Dinner     2    -6.30
       178        9.60  4.00    Yes   Sun  Dinner     2    -5.60
       92         5.75  1.00    Yes   Fri  Dinner     2    -4.75
       172        7.25  5.15    Yes   Sun  Dinner     2    -2.10
       67         3.07  1.00    Yes   Sat  Dinner     1    -2.07

In [30]:
result = tips.groupby('smoker')['tip_pct'].describe()

In [31]:
result

,count,mean,std,min,25%,50%,75%,max
smoker,,,,,,,,
No,151.0,-16.196424,7.166287,-43.17,-19.245,-14.52,-10.98,-5.51
Yes,93.0,-17.747634,9.229335,-41.85,-23.180,-14.65,-11.51,-2.07


### 3.1压缩分组键

In [32]:
tips.groupby('smoker', group_keys=False).apply(top)

,total_bill,tip,smoker,day,time,size,tip_pct
145,8.35,1.50,No,Thur,Lunch,2,-6.85
6,8.77,2.00,No,Sun,Dinner,2,-6.77
111,7.25,1.00,No,Sat,Dinner,1,-6.25
195,7.56,1.44,No,Thur,Lunch,2,-6.12
149,7.51,2.00,No,Thur,Lunch,2,-5.51
218,7.74,1.44,Yes,Sat,Dinner,2,-6.30
178,9.60,4.00,Yes,Sun,Dinner,2,-5.60
92,5.75,1.00,Yes,Fri,Dinner,2,-4.75
172,7.25,5.15,Yes,Sun,Dinner,2,-2.10
67,3.07,1.00,Yes,Sat,Dinner,1,-2.07


### 3.2分位数与桶分析

In [33]:
frame = pd.DataFrame({'data1': np.random.randn(1000),
                     'data2': np.random.randn(1000)})

In [34]:
quartiles = pd.cut(frame.data1, 4)

In [35]:
quartiles[:10]

0     (0.525, 2.197]
1    (-1.148, 0.525]
2    (-1.148, 0.525]
3    (-1.148, 0.525]
4    (-1.148, 0.525]
5     (0.525, 2.197]
6     (0.525, 2.197]
7     (0.525, 2.197]
8    (-1.148, 0.525]
9     (0.525, 2.197]
Name: data1, dtype: category
Categories (4, interval[float64]): [(-2.826, -1.148] < (-1.148, 0.525] < (0.525, 2.197] < (2.197, 3.869]]

## 4数据透视表与交叉表

In [36]:
tips.pivot_table(index = ['day', 'smoker'])

size       tip    tip_pct  total_bill
day  smoker                                           
Fri  No      2.250000  2.812500 -15.607500   18.420000
     Yes     2.066667  2.714000 -14.099333   16.813333
Sat  No      2.555556  3.102889 -16.558889   19.661778
     Yes     2.476190  2.875476 -18.401190   21.276667
Sun  No      2.929825  3.167895 -17.338772   20.506667
     Yes     2.578947  3.516842 -20.603158   24.120000
Thur No      2.488889  2.673778 -14.439333   17.113111
     Yes     2.352941  3.030000 -16.160588   19.190588

In [37]:
tips.pivot_table(['tip_pct', 'size'], index = ['time', 'day'],
                columns = 'smoker')

size              tip_pct           
smoker             No       Yes         No        Yes
time   day                                           
Dinner Fri   2.000000  2.222222 -16.483333 -16.803333
       Sat   2.555556  2.476190 -16.558889 -18.401190
       Sun   2.929825  2.578947 -17.338772 -20.603158
       Thur  2.000000       NaN -15.780000        NaN
Lunch  Fri   3.000000  1.833333 -12.980000 -10.043333
       Thur  2.500000  2.352941 -14.408864 -16.160588